In [14]:
import sklearn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
from matplotlib import pyplot as plt
from sklearn import preprocessing

In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [3]:
data = pd.read_csv('data/IMDB Dataset.csv')

In [4]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
vect = CountVectorizer(max_features=10000).fit(data['review'])
#vect = TfidfVectorizer(max_features=10000).fit(data['review'])
#  (max_features=2500, min_df=7, max_df=0.8, stop_words=stopwords.words('english')
X = vect.transform(data['review'])

In [6]:
#print(vect.vocabulary_)
print(len(vect.stop_words_))

91895


In [7]:
y = data['sentiment'].apply(lambda x: 1 if x == 'positive' else 0)

In [8]:
feature_names = vect.get_feature_names()
print("Number of features: {}".format(len(feature_names)))
print("First 20 features:\n{}".format(feature_names[:100]))
print("Features 5010 to 5030:\n{}".format(feature_names[5010:5030]))
print("Every 2000th feature:\n{}".format(feature_names[::2000]))

Number of features: 10000
First 20 features:
['00', '000', '007', '10', '100', '1000', '101', '11', '12', '13', '13th', '14', '15', '150', '16', '17', '18', '18th', '19', '1920s', '1930', '1930s', '1931', '1932', '1933', '1934', '1936', '1938', '1939', '1940', '1940s', '1941', '1942', '1943', '1944', '1945', '1946', '1947', '1948', '1949', '1950', '1950s', '1951', '1952', '1953', '1955', '1956', '1957', '1958', '1959', '1960', '1960s', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970', '1970s', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1980s', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1990s', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '19th', '1st', '20', '200', '2000', '2001', '2002']
Features 5010 to 5030:
['kinky', 'kinnear', 'kinski', 'kirk', 'kiss', 'kisses', 'kissing', 'kitchen', 'kite', 'kitty', 'klaus', 'klein', 'kline', 'knack', 'knee', 'knees', 'knew

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
#from sklearn import tree
#clf = tree.DecisionTreeClassifier(max_depth=25, min_samples_leaf=10)
#clf = clf.fit(X_train, y_train)

In [11]:
#from sklearn import svm
#clf = svm.SVC(C=1, kernel='linear')
#clf = clf.fit(X_train, y_train)

In [16]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(max_iter=5000)
clf = clf.fit(X_train, y_train)

In [37]:
#from sklearn.naive_bayes import GaussianNB
#clf = GaussianNB()
#clf = clf.fit(X_train.toarray(), y_train)

In [17]:
y_pred = clf.predict(X_test.toarray())
y_train_pred = clf.predict(X_train.toarray())

In [18]:
test_accuracy = accuracy_score(y_test, y_pred)
train_accuracy = accuracy_score(y_train, y_train_pred)

In [19]:
print("accuracy on training set = {:1.1f}%".format(100*train_accuracy))
print("accuracy on test set\t = {:1.1f}%".format(100*test_accuracy))

accuracy on training set = 97.8%
accuracy on test set	 = 87.5%


In [20]:
try:
    indices = np.argsort(clf.coef_[0])
    feature_names = np.array(vect.get_feature_names())[indices]
    neg_unigrams = feature_names[:100]
    print('Words associated with negative reviews:')
    print(neg_unigrams)
    pos_unigrams = feature_names[-100:-1]
    print('\n')
    print('Words associated with positive reviews:')
    print(pos_unigrams)
except:
    print('Print classifier does not have coeff attribute')

Words associated with negative reviews:
['waste' 'mst3k' 'disappointment' 'unremarkable' 'forgettable' 'worst'
 'uninteresting' 'olds' 'stinker' 'lifeless' 'alright' 'baldwin' 'rainy'
 'leonard' 'sticking' 'tunnel' 'motions' 'tiresome' 'prolonged'
 'awfulness' 'disappointing' 'endlessly' 'controls' 'wasting' 'distinctly'
 'poorly' 'laughable' 'unfunny' 'tedious' 'lousy' 'mediocre'
 'unintentionally' 'lacks' 'dire' 'coincidences' 'inappropriate'
 'obnoxious' 'mildly' 'annoyed' 'boredom' 'appalling' 'bore' 'unwatchable'
 'ethan' 'awful' 'behave' 'costs' 'miscast' 'uninspired' 'feed' 'yep'
 'boring' 'dreadful' 'smug' 'fails' 'ludicrous' 'jefferson' 'jigsaw'
 'nope' 'turkey' 'bland' 'unlikeable' 'neal' 'slows' 'victims' 'messy'
 'clone' 'cannibal' 'manipulative' 'dumber' 'credibility' 'revolting'
 'avoid' 'amateurish' 'stares' 'hype' 'skip' 'endless' 'lame' 'rourke'
 'stereotyped' 'horrible' 'recycled' 'senseless' 'mia' 'christians'
 'dramatically' 'save' 'belongs' 'unpleasant' 'generous' 

In [16]:
sentiment = lambda x: 'positive' if x == 1 else 'negative'

In [17]:
review = ['this movie was shit and crap at the same time']
x_rev = vect.transform(review)
y_pred = clf.predict(x_rev.toarray())
print('Sentiment of review: {}'.format(sentiment(y_pred)))

Sentiment of review: negative
